In [1]:
from collections import Counter
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pickle
from pulp import *
from sklearn.linear_model import LinearRegression
import pandas as pd
from datetime import timedelta,time,datetime
import numpy as np
import matplotlib.pyplot as plt
import copy
from sklearn.utils import shuffle
import datetime as DateTime
import pickle
from utilities import *
from sklearn.ensemble import RandomForestRegressor
from random import sample
from environment import *


# Set field values

In [2]:
intervalLength=5
surplusVol=0.95
save=False

In [3]:
priceDf=pd.read_csv("data sets/day_ahead_price_pre_proc.csv")
intraDf=pd.read_csv("data sets/intra_day_price_pre_proc.csv")

In [4]:
preprocessedLoc="data sets/preproc_charge_sessions.csv"
df = pd.read_csv(preprocessedLoc,parse_dates=["Started", "Ended",'start_time','EndCharge','arrival_time', 'due_time','latest_start_time'])
fileName=str(surplusVol).replace('.','_')
demand1= pickle.load( open("source_solution/source_orig_"+fileName, "rb" ))

# Functionalities


In [5]:
def min_max_volume(job,time,intervalLength):
    dueTime=job['duetime_no']
    maxPower=job['MaxPower(KW)']
    maxTime=max(dueTime-time,1)
    minTime=1
    maxVolume=maxTime*np.round(maxPower*intervalLength/60+0.0005,3)
    minVolume=np.round(intervalLength/60*maxPower+0.0005,3)
    return minVolume,maxVolume



In [6]:
def estimate_duration(pendingJobs,time,trainDf,intervalLength,estimate=None,reg=None):
    newJobs=[]
    failSamp=0
    if type(estimate)==type(None):
        for job in pendingJobs:
            real=np.round(job['TotalEnergy(KWh)']+0.0005,3)
            job['estimatedVolume']=real
            newJobs.append(job)
    elif estimate=='SL':
        toPred=pd.DataFrame(pendingJobs)[['arrivaltime_no','duetime_no','time_till_due','MaxPower(KW)','weekday']]
        predictions=reg.predict(toPred)
        for job,pred in zip(pendingJobs,predictions):
            estVol=pred
            minVolume,maxVolume=min_max_volume(job,time,intervalLength)
            if estVol>maxVolume:
                failSamp+=1
                estVol=maxVolume
            if estVol<minVolume:
                failSamp+=1
                estVol=minVolume
            job['estimatedVolume']=estVol
            newJobs.append(job)
    elif estimate == 'sample':
        for job in pendingJobs:
            minVolume,maxVolume=min_max_volume(job,time,intervalLength)
            sampleVolume=copy.deepcopy(trainDf['TotalEnergy(KWh)'].values)
            sampleVolume-=job['chargedVolume']
            sampleVolume=[i for i in sampleVolume if i>0] 
            
            if len(sampleVolume)==0:
                estVol=np.mean(trainDf['TotalEnergy(KWh)'].values)
                if estVol>maxVolume:
                    estVol=maxVolume
            else:
                estVol=sample(sampleVolume,1)[0]
                if estVol>maxVolume:
                    estVol=maxVolume 
            job['estimatedVolume']=estVol
            newJobs.append(job)
            
    return newJobs,True

In [7]:
def sample_jobs(env,time,intervalLength):
    df=copy.deepcopy(env.trainDf)
    
    #construct data to sample from
    days=list(df['days_for_aggregation'].unique())
    df=df[df['weekday']==env.weekday]
    avgJobs=df[(df['arrivaltime_no']>time)&(df['arrivaltime_no']<(time+(12*(60/intervalLength))))]
    
    #determine sample size
    sampleSize=int(len(avgJobs)/len(list(df['days_for_aggregation'].unique())))
    
    #if possible sample jobs
    if len(avgJobs)>10 and sampleSize>0:
        sample=avgJobs.sample(sampleSize,replace=True)
    else:
        sample=pd.DataFrame()
        print("not enough to sample")
    return sample

In [8]:
def sample_target(quedJobs,intervalLength,time=None,demand=None,sampScheds=None,maxTime=10):
    jobNumbers  = range(len(quedJobs))
    timeList=list(range(time,min(time+(1*int(60/intervalLength)),287)))
    samples=range(len(sampScheds))

    #initialization
    varSched = pulp.LpVariable.dicts("if_i_charged_at_j", ((i, j) for i in jobNumbers for j in timeList),cat='Binary')
    varError = pulp.LpVariable.dicts("if_i_error_at_j", ((i, j,e,s) for i in jobNumbers for j in timeList for e in [0,1] for s in samples))
    model = pulp.LpProblem("scheduling_model", pulp.LpMinimize)
    
    #objective:
    model+=lpSum(varError[(i, j,1,s)] for i in jobNumbers for j in timeList for s in samples) + len(sampScheds)*lpSum(varError[(i, j,0,s)] for i in jobNumbers for j in timeList for s in samples)
   
    #capacity restriction
    for j in timeList:
        model+=lpSum(varSched[(jobNo,j)]*quedJobs['MaxPower(KW)'].values[jobNo] for jobNo in jobNumbers) <= demand[j]
        
    #restrictions absolute error
    for i in jobNumbers:
        for j in timeList:
            for s in samples:
                model+=  varSched[(i,j)] - sampScheds[s][i][j]  +varError[(i,j,0,s)] - varError[(i,j,1,s)] == 0
                model += varError[(i,j,0,s)] >= 0
                model += varError[(i,j,1,s)] >= 0
    
    #restrictions startime, endtime
    jobNo=0
    for arrivTime,dueTime  in zip(quedJobs['arrivaltime_no'],quedJobs['duetime_no']):
        infeasTimes=np.setdiff1d(time,range(int(arrivTime),int(dueTime)))
        j=0
        for t in infeasTimes:

            if t-time>=0:
                j+=1
                try:
                    model += varSched[(jobNo,t)]==0
                except:
                    continue
        jobNo+=1

    model.solve(PULP_CBC_CMD(fracGap = 0.001, maxSeconds = maxTime, threads = None))
    print("Status:", LpStatus[model.status])
    
    #construct schedule
    schedule=np.zeros((len(jobNumbers),24*int(60/intervalLength)))
    rows=[]
    listIsCharged=[b for b in model.variables() if b.varValue>0 and "if_i_charged_at_j" in b.name]
    for i in listIsCharged:
        split=str(i).split('(')[1]
        row=split.split(',')[0]
        rows.append(int(row))
        column=split.split(',')[1][1:-1]
        schedule[int(row)][int(column)]=1

    return schedule


In [9]:
def opt_indiv_sample(quedJobs,intervalLength,time=None,demand=None,maxTime=10):
    numberOfJobs  = range(len(quedJobs))
    
    #optimize from time onwards
    timeNo=time
    timeList=list(range(time,min(time+(12*int(60/intervalLength)),287)))
    
    #initializations
    varSched = pulp.LpVariable.dicts("if_i_charged_at_j", ((i, j) for i in numberOfJobs for j in timeList),cat='Binary')
    model = pulp.LpProblem("scheduling_model", pulp.LpMaximize)
        
    #objective:
    model+=lpSum(lpSum(varSched[(jobNo,j)]*quedJobs['MaxPower(KW)'].values[jobNo] for jobNo in numberOfJobs) for j in timeList)
    
    #restrictions for objective
    for j in timeList:
        model+= lpSum(varSched[(jobNo,j)]*quedJobs['MaxPower(KW)'].values[jobNo] for jobNo in numberOfJobs) <= demand[j]

    #restrictions startTime, endTIme
    jobNo=0
    for arrivTime,dueTime  in zip(quedJobs['arrivaltime_no'],quedJobs['duetime_no']):
        infeasTimes=np.setdiff1d(timeList,range(int(arrivTime),int(dueTime)))
        j=0
        for t in infeasTimes:
            if t in timeList:
                model += varSched[(jobNo,t)]==0
            else:
                continue
        jobNo+=1
        
    #restrictions total volume
    jobNo=0
    for energy,maxPower,dueTime in zip(quedJobs['estimatedVolume'],quedJobs['MaxPower(KW)'],quedJobs['duetime_no']):
        correction=np.round((intervalLength/60)*maxPower-0.0005,3)
        if energy-correction<0:
            correction=0
            energy=0
        model += lpSum([varSched[item]*np.round((intervalLength/60)*maxPower+0.0005,3) for item in varSched.keys() if item[0]==jobNo]) <= energy
        jobNo+=1
    
    model.solve(PULP_CBC_CMD(fracGap = 0.05, maxSeconds = 80, threads = None))
    print("Status:", LpStatus[model.status])
    
    #construct schedule
    schedule=np.zeros((len(numberOfJobs),24*int(60/intervalLength)))
    rows=[]
    listIsCharged=[b for b in model.variables() if b.varValue>0 and "if_i_charged_at_j" in b.name]

    for i in listIsCharged:
        split=str(i).split('(')[1]
        row=split.split(',')[0]
        rows.append(int(row))
        column=split.split(',')[1][1:-1]
        schedule[int(row)][int(column)]=1

    return schedule

In [10]:
def convert_sched_to_position(schedule,pendingJobs):
    i=0
    for job in range(schedule.shape[0]):
        timeStep=next((i for i, x in enumerate(schedule[job][:]) if x), 0)
        if sum(schedule[job,:])==0:
            timeStep=len(schedule[job,:])
        jobQ=pendingJobs[i]
        jobQ['allocatedPos']=timeStep
        pendingJobs[i]=jobQ
        i+=1
    return pendingJobs

def sample_average_approximation(env,pendingJobs,time,samps=3,estimate=None,reg=None):
    jobsAreSorted=False
    
    #select sample dataframe
    sampleDays=list(env.trainDf[env.trainDf['weekday']==env.weekday]['days_for_aggregation'].unique())
    
    
    sampleScheds=[]
    #sample I days of N jobs
    for day in range(samps):
       
        sampleJobs=sample_jobs(env,time,intervalLength)
        if len(sampleJobs)==0:
            break
        else:
             #estimate duration
            pendingJobs,status=estimate_duration(pendingJobs,time,env.trainDf,intervalLength,estimate=estimate,reg=reg)
            if not status:
                break
            #convert pending jobs to pandas
            quedJobs=pd.DataFrame(pendingJobs)
            
            sampleJobs['chargedVolume']=0
            sampleJobs['estimatedVolume']=sampleJobs['TotalEnergy(KWh)']
            sampleJobs['allocatedPos']=0
            sampleJobs['target']=0
            sampleJobs['orig_energ(KWh)']=sampleJobs['TotalEnergy(KWh)']
            sampleJobs=sampleJobs[list(quedJobs.keys())]
            
            #merge sample with queued jobs
            jobsToSchedule=copy.deepcopy(quedJobs)
            jobsToSchedule=jobsToSchedule.append(sampleJobs,ignore_index=True)
            jobsToSchedule=jobsToSchedule.reset_index()
    
            schedule=opt_indiv_sample(jobsToSchedule,intervalLength,
                                                  demand=env.demand,
                                                  time=time,maxTime=30)
            
            #select schedule of queud jobs
            schedule=schedule[0:len(quedJobs):][:]
            
            sampleScheds.append(schedule)

    if len(sampleScheds)==samps:
        #calculate optimal sample target
        schedule=sample_target(quedJobs,intervalLength,demand=env.demand,time=time,
                                                  sampScheds=sampleScheds,maxTime=5)
        
        #convert schedule to postion
        pendingJobs=convert_sched_to_position(schedule,pendingJobs)
        
        pendingJobs=sorted(pendingJobs, key=lambda tup: tup['allocatedPos'])
        jobsAreSorted=True
    if not jobsAreSorted:
        print('not sorted')
   
    return pendingJobs

In [11]:
#predict duration
def train_rf_regression(trainDf):
    print("train regressor")
    dataSupv=trainDf[['arrivaltime_no','duetime_no','time_till_due','MaxPower(KW)','weekday','TotalEnergy(KWh)']]
    X=dataSupv.drop('TotalEnergy(KWh)',axis=1)
    y=dataSupv['TotalEnergy(KWh)']
    reg = RandomForestRegressor(min_samples_split=20,min_samples_leaf=20).fit(X[0:int(X.shape[0]*0.7)],y[0:int(X.shape[0]*0.7)])
    print('train performance: ',reg.score(X[0:int(X.shape[0]*0.7)],y[0:int(X.shape[0]*0.7)]))
    print('test performance: ',reg.score(X[int(X.shape[0]*0.7):], y[int(X.shape[0]*0.7):]))
    reg = RandomForestRegressor(min_samples_split=20,min_samples_leaf=20).fit(X,y)
    return reg

In [12]:
def schedule_to_job(optSched,pendingJobs,timeNo):
    newJobs=[]
    for i,job in enumerate(pendingJobs):
        if optSched[i][timeNo]==1:
            job['target']=1
        else:
            job['target']=0
        newJobs.append(job)

    return newJobs

In [13]:
def run(env,episodes,intervalLength,strategy,verbose=1,dataSupv=None):
    totalCosts=0
    totalUnchargedCapacity=0
    totalCharged=0
    noIntraCosts=0
    totalEnergy=0
    reg=None
    for e in range(episodes):
        dayCharged=0
        uncharged=0
        pendingJobs=[]
        
        #start with first job
        jobs,done,time=env.step()
        for job in jobs:
            pendingJobs.append(job)
        
        done=False
        leftoverEnergy=0
        doneJobs=0
        
        pendingJobs,noUnfeasible,leftover=env.get_feasible_jobs(pendingJobs,time)
        uncharged+=noUnfeasible
        leftoverEnergy+=leftover
        
        
        #take steps untill env return False
        ite=-1
        while not done:
            ite+=1
            if verbose==1:
                print("iteraction no: ",ite,"out of: ",str(12*24))
            if strategy=='opt_disp':
                pendingJobs=sorted(pendingJobs, key=lambda tup: (-max(0,(tup['TotalEnergy(KWh)']/tup['MaxPower(KW)'])-((tup['duetime_no']-time)/12)),tup['duetime_no']))
            if strategy=='priority':
                pendingJobs=sorted(pendingJobs, key=lambda tup: tup['duetime_no'])
            if strategy=="SL":
                if len(pendingJobs)>0:
                    pendingJobs=supervised_prediction(env,pendingJobs,time,env.timeIntervals)
            if strategy=="SAA_Samp":
                if len(pendingJobs)>0:
                    pendingJobs=sample_average_approximation(env,pendingJobs,time,estimate='sample')
            if strategy=="SAA_SL":
                if type(reg)==type(None):
                    reg=train_rf_regression(env.trainDf)
                if len(pendingJobs)>0:
                    pendingJobs=sample_average_approximation(env,pendingJobs,time,estimate='SL',reg=reg)
            if strategy=="SAA":
                if len(pendingJobs)>0:
                    pendingJobs=sample_average_approximation(env,pendingJobs,time)
            if strategy=='length':
                pendingJobs=sorted(pendingJobs, key=lambda tup: tup['duration'],reverse=True)
            if strategy=='arrival':
                pendingJobs=sorted(pendingJobs, key=lambda tup: tup['arrivaltime_no'],reverse=True)
                
            if strategy=="ilp":
                if ite==0:
                    schedule=iLPOptSchedule(env.episode,intervalLength,
                                                  env.demand,
                                                  priceOpt=False,
                                                          allocOpt=True,day=env.currentEpisode,maxTime=50)
                    power=np.array(env.episode['MaxPower(KW)'])/12

                    volumes=np.sum(schedule*power[:, np.newaxis],axis=1)
                    print(sum(volumes.flatten()))
                    print(schedule)

                pendingJobs=schedule_to_job(schedule,pendingJobs,time)
                
            storeJobs=[]
            availableCapacity=True
            #plan jobs untill all jobs are planned or there is no more capacity
            
            if strategy!='ilp' and len(pendingJobs)>0:
                pendingJobs,charged=env.process_queue(time,pendingJobs)
                dayCharged+=charged
            else:
                chargedJobsies=0
                for i in range(len(pendingJobs)):
                    availableCapacity=env.check_available_capacity(time,pendingJobs[i])
                    if pendingJobs[i]['target']==1 and availableCapacity:
                        job,charged=env.plan_job(pendingJobs[i],time)
                        dayCharged+=charged
                        pendingJobs[i]=job
                        chargedJobsies+=1
            
            jobs,done,time=env.step()
            
            for job in jobs:
                pendingJobs.append(job)
            if strategy!='ilp':
                pendingJobs,noUnfeasible,leftover=env.get_feasible_jobs(pendingJobs,time)
                uncharged+=noUnfeasible
                leftoverEnergy+=leftover
        totalCharged+=dayCharged
        totalUnchargedCapacity+=leftoverEnergy
        costs,costsWithoutIntra=env.calculate_costs()
        noIntraCosts+=costsWithoutIntra
        totalCosts+=costs
        totalDem=int(totalDemand(env.demand,intervalLength))
        totalEnergy+=totalDem
        if verbose==1:
            print("demand left:___",int(totalDemand(env.demandLeft,intervalLength)),"orig demand___",totalDem,"total  charged___",int(dayCharged))
            print("not charged:___",uncharged,"not ended:___",len(pendingJobs),"out of: ",len(env.episode))
            print("uncharged capacity:___",int(leftoverEnergy))
            print("costs:___",costs)
            print('\n')
        
        if done:
            env.reset()
    print("cost/charged capacity(KWh)__:",np.round(totalCosts/totalCharged,4),
          "cost/charged capacity no intraday__:",np.round(noIntraCosts/totalCharged,3),
          "percentage charged capacity__:",np.round(totalCharged/totalEnergy,2),
            "volume charged__:",int(totalCharged))
    return uncharged,int(totalCharged),np.round(totalCosts/totalCharged,4)
    
    
    

# Algorithms

In [15]:
results={}
trainDf=df[df['days_for_aggregation']<35]

In [71]:
#total demand
maximumVolume=df[df['days_for_aggregation']>=35]['TotalEnergy(KWh)'].sum()
results['totalDemand']=(maximumVolume)

In [17]:
#ilp solution
env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"ilp",verbose=1)
results['ilp']=(capacity,costs)

iteraction no:  0 out of:  288
Status: Optimal
2438.4400000000005
[[0. 0. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
iteraction no:  1 out of:  288
iteraction no:  2 out of:  288
iteraction no:  3 out of:  288
iteraction no:  4 out of:  288
iteraction no:  5 out of:  288
iteraction no:  6 out of:  288
iteraction no:  7 out of:  288
iteraction no:  8 out of:  288
iteraction no:  9 out of:  288
iteraction no:  10 out of:  288
iteraction no:  11 out of:  288
iteraction no:  12 out of:  288
iteraction no:  13 out of:  288
iteraction no:  14 out of:  288
iteraction no:  15 out of:  288
iteraction no:  16 out of:  288
iteraction no:  17 out of:  288
iteraction no:  18 out of:  288
iteraction no:  19 out of:  288
iteraction no:  20 out of:  288
iteraction no:  21 out of:  288
iteraction no:  22 out of:  288
iteraction no:  23 out of:  288
iteraction no:  24 out of:  288
iteraction no:  25 

KeyboardInterrupt: 

In [33]:
#ilp solution
env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"ilp",verbose=1)
results['ilp']=(capacity,costs)

iteraction no:  0 out of:  288
Status: Optimal
2438.4400000000005
[[0. 0. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
iteraction no:  1 out of:  288
iteraction no:  2 out of:  288
iteraction no:  3 out of:  288
iteraction no:  4 out of:  288
iteraction no:  5 out of:  288
iteraction no:  6 out of:  288
iteraction no:  7 out of:  288
iteraction no:  8 out of:  288
iteraction no:  9 out of:  288
iteraction no:  10 out of:  288
iteraction no:  11 out of:  288
iteraction no:  12 out of:  288
iteraction no:  13 out of:  288
iteraction no:  14 out of:  288
iteraction no:  15 out of:  288
iteraction no:  16 out of:  288
iteraction no:  17 out of:  288
iteraction no:  18 out of:  288
iteraction no:  19 out of:  288
iteraction no:  20 out of:  288
iteraction no:  21 out of:  288
iteraction no:  22 out of:  288
iteraction no:  23 out of:  288
iteraction no:  24 out of:  288
iteraction no:  25 

iteraction no:  246 out of:  288
iteraction no:  247 out of:  288
iteraction no:  248 out of:  288
iteraction no:  249 out of:  288
iteraction no:  250 out of:  288
iteraction no:  251 out of:  288
iteraction no:  252 out of:  288
iteraction no:  253 out of:  288
iteraction no:  254 out of:  288
iteraction no:  255 out of:  288
iteraction no:  256 out of:  288
iteraction no:  257 out of:  288
iteraction no:  258 out of:  288
iteraction no:  259 out of:  288
iteraction no:  260 out of:  288
iteraction no:  261 out of:  288
iteraction no:  262 out of:  288
iteraction no:  263 out of:  288
iteraction no:  264 out of:  288
iteraction no:  265 out of:  288
iteraction no:  266 out of:  288
iteraction no:  267 out of:  288
iteraction no:  268 out of:  288
iteraction no:  269 out of:  288
iteraction no:  270 out of:  288
iteraction no:  271 out of:  288
iteraction no:  272 out of:  288
iteraction no:  273 out of:  288
iteraction no:  274 out of:  288
iteraction no:  275 out of:  288
iteraction

iteraction no:  202 out of:  288
iteraction no:  203 out of:  288
iteraction no:  204 out of:  288
iteraction no:  205 out of:  288
iteraction no:  206 out of:  288
iteraction no:  207 out of:  288
iteraction no:  208 out of:  288
iteraction no:  209 out of:  288
iteraction no:  210 out of:  288
iteraction no:  211 out of:  288
iteraction no:  212 out of:  288
iteraction no:  213 out of:  288
iteraction no:  214 out of:  288
iteraction no:  215 out of:  288
iteraction no:  216 out of:  288
iteraction no:  217 out of:  288
iteraction no:  218 out of:  288
iteraction no:  219 out of:  288
iteraction no:  220 out of:  288
iteraction no:  221 out of:  288
iteraction no:  222 out of:  288
iteraction no:  223 out of:  288
iteraction no:  224 out of:  288
iteraction no:  225 out of:  288
iteraction no:  226 out of:  288
iteraction no:  227 out of:  288
iteraction no:  228 out of:  288
iteraction no:  229 out of:  288
iteraction no:  230 out of:  288
iteraction no:  231 out of:  288
iteraction

iteraction no:  156 out of:  288
iteraction no:  157 out of:  288
iteraction no:  158 out of:  288
iteraction no:  159 out of:  288
iteraction no:  160 out of:  288
iteraction no:  161 out of:  288
iteraction no:  162 out of:  288
iteraction no:  163 out of:  288
iteraction no:  164 out of:  288
iteraction no:  165 out of:  288
iteraction no:  166 out of:  288
iteraction no:  167 out of:  288
iteraction no:  168 out of:  288
iteraction no:  169 out of:  288
iteraction no:  170 out of:  288
iteraction no:  171 out of:  288
iteraction no:  172 out of:  288
iteraction no:  173 out of:  288
iteraction no:  174 out of:  288
iteraction no:  175 out of:  288
iteraction no:  176 out of:  288
iteraction no:  177 out of:  288
iteraction no:  178 out of:  288
iteraction no:  179 out of:  288
iteraction no:  180 out of:  288
iteraction no:  181 out of:  288
iteraction no:  182 out of:  288
iteraction no:  183 out of:  288
iteraction no:  184 out of:  288
iteraction no:  185 out of:  288
iteraction

iteraction no:  114 out of:  288
iteraction no:  115 out of:  288
iteraction no:  116 out of:  288
iteraction no:  117 out of:  288
iteraction no:  118 out of:  288
iteraction no:  119 out of:  288
iteraction no:  120 out of:  288
iteraction no:  121 out of:  288
iteraction no:  122 out of:  288
iteraction no:  123 out of:  288
iteraction no:  124 out of:  288
iteraction no:  125 out of:  288
iteraction no:  126 out of:  288
iteraction no:  127 out of:  288
iteraction no:  128 out of:  288
iteraction no:  129 out of:  288
iteraction no:  130 out of:  288
iteraction no:  131 out of:  288
iteraction no:  132 out of:  288
iteraction no:  133 out of:  288
iteraction no:  134 out of:  288
iteraction no:  135 out of:  288
iteraction no:  136 out of:  288
iteraction no:  137 out of:  288
iteraction no:  138 out of:  288
iteraction no:  139 out of:  288
iteraction no:  140 out of:  288
iteraction no:  141 out of:  288
iteraction no:  142 out of:  288
iteraction no:  143 out of:  288
iteraction

iteraction no:  67 out of:  288
iteraction no:  68 out of:  288
iteraction no:  69 out of:  288
iteraction no:  70 out of:  288
iteraction no:  71 out of:  288
iteraction no:  72 out of:  288
iteraction no:  73 out of:  288
iteraction no:  74 out of:  288
iteraction no:  75 out of:  288
iteraction no:  76 out of:  288
iteraction no:  77 out of:  288
iteraction no:  78 out of:  288
iteraction no:  79 out of:  288
iteraction no:  80 out of:  288
iteraction no:  81 out of:  288
iteraction no:  82 out of:  288
iteraction no:  83 out of:  288
iteraction no:  84 out of:  288
iteraction no:  85 out of:  288
iteraction no:  86 out of:  288
iteraction no:  87 out of:  288
iteraction no:  88 out of:  288
iteraction no:  89 out of:  288
iteraction no:  90 out of:  288
iteraction no:  91 out of:  288
iteraction no:  92 out of:  288
iteraction no:  93 out of:  288
iteraction no:  94 out of:  288
iteraction no:  95 out of:  288
iteraction no:  96 out of:  288
iteraction no:  97 out of:  288
iteracti

iteraction no:  19 out of:  288
iteraction no:  20 out of:  288
iteraction no:  21 out of:  288
iteraction no:  22 out of:  288
iteraction no:  23 out of:  288
iteraction no:  24 out of:  288
iteraction no:  25 out of:  288
iteraction no:  26 out of:  288
iteraction no:  27 out of:  288
iteraction no:  28 out of:  288
iteraction no:  29 out of:  288
iteraction no:  30 out of:  288
iteraction no:  31 out of:  288
iteraction no:  32 out of:  288
iteraction no:  33 out of:  288
iteraction no:  34 out of:  288
iteraction no:  35 out of:  288
iteraction no:  36 out of:  288
iteraction no:  37 out of:  288
iteraction no:  38 out of:  288
iteraction no:  39 out of:  288
iteraction no:  40 out of:  288
iteraction no:  41 out of:  288
iteraction no:  42 out of:  288
iteraction no:  43 out of:  288
iteraction no:  44 out of:  288
iteraction no:  45 out of:  288
iteraction no:  46 out of:  288
iteraction no:  47 out of:  288
iteraction no:  48 out of:  288
iteraction no:  49 out of:  288
iteracti

iteraction no:  284 out of:  288
iteraction no:  285 out of:  288
iteraction no:  286 out of:  288
iteraction no:  287 out of:  288
demand left:___ 149 orig demand___ 1891 total  charged___ 1742
not charged:___ 0 not ended:___ 255 out of:  255
uncharged capacity:___ 0
costs:___ 79.506


iteraction no:  0 out of:  288
Status: Optimal
1552.4275000000002
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
iteraction no:  1 out of:  288
iteraction no:  2 out of:  288
iteraction no:  3 out of:  288
iteraction no:  4 out of:  288
iteraction no:  5 out of:  288
iteraction no:  6 out of:  288
iteraction no:  7 out of:  288
iteraction no:  8 out of:  288
iteraction no:  9 out of:  288
iteraction no:  10 out of:  288
iteraction no:  11 out of:  288
iteraction no:  12 out of:  288
iteraction no:  13 out of:  288
iteraction no:  14 out of:  288
iteraction no:  15 out of:  288
iteraction no:  16 

iteraction no:  238 out of:  288
iteraction no:  239 out of:  288
iteraction no:  240 out of:  288
iteraction no:  241 out of:  288
iteraction no:  242 out of:  288
iteraction no:  243 out of:  288
iteraction no:  244 out of:  288
iteraction no:  245 out of:  288
iteraction no:  246 out of:  288
iteraction no:  247 out of:  288
iteraction no:  248 out of:  288
iteraction no:  249 out of:  288
iteraction no:  250 out of:  288
iteraction no:  251 out of:  288
iteraction no:  252 out of:  288
iteraction no:  253 out of:  288
iteraction no:  254 out of:  288
iteraction no:  255 out of:  288
iteraction no:  256 out of:  288
iteraction no:  257 out of:  288
iteraction no:  258 out of:  288
iteraction no:  259 out of:  288
iteraction no:  260 out of:  288
iteraction no:  261 out of:  288
iteraction no:  262 out of:  288
iteraction no:  263 out of:  288
iteraction no:  264 out of:  288
iteraction no:  265 out of:  288
iteraction no:  266 out of:  288
iteraction no:  267 out of:  288
iteraction

In [34]:
#priority first
env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"priority",verbose=1)
results['prio first']=(capacity,costs)

iteraction no:  0 out of:  288
iteraction no:  1 out of:  288
iteraction no:  2 out of:  288
iteraction no:  3 out of:  288
iteraction no:  4 out of:  288
iteraction no:  5 out of:  288
iteraction no:  6 out of:  288
iteraction no:  7 out of:  288
iteraction no:  8 out of:  288
iteraction no:  9 out of:  288
iteraction no:  10 out of:  288
iteraction no:  11 out of:  288
iteraction no:  12 out of:  288
iteraction no:  13 out of:  288
iteraction no:  14 out of:  288
iteraction no:  15 out of:  288
iteraction no:  16 out of:  288
iteraction no:  17 out of:  288
iteraction no:  18 out of:  288
iteraction no:  19 out of:  288
iteraction no:  20 out of:  288
iteraction no:  21 out of:  288
iteraction no:  22 out of:  288
iteraction no:  23 out of:  288
iteraction no:  24 out of:  288
iteraction no:  25 out of:  288
iteraction no:  26 out of:  288
iteraction no:  27 out of:  288
iteraction no:  28 out of:  288
iteraction no:  29 out of:  288
iteraction no:  30 out of:  288
iteraction no:  31

iteraction no:  257 out of:  288
iteraction no:  258 out of:  288
iteraction no:  259 out of:  288
iteraction no:  260 out of:  288
iteraction no:  261 out of:  288
iteraction no:  262 out of:  288
iteraction no:  263 out of:  288
iteraction no:  264 out of:  288
iteraction no:  265 out of:  288
iteraction no:  266 out of:  288
iteraction no:  267 out of:  288
iteraction no:  268 out of:  288
iteraction no:  269 out of:  288
iteraction no:  270 out of:  288
iteraction no:  271 out of:  288
iteraction no:  272 out of:  288
iteraction no:  273 out of:  288
iteraction no:  274 out of:  288
iteraction no:  275 out of:  288
iteraction no:  276 out of:  288
iteraction no:  277 out of:  288
iteraction no:  278 out of:  288
iteraction no:  279 out of:  288
iteraction no:  280 out of:  288
iteraction no:  281 out of:  288
iteraction no:  282 out of:  288
iteraction no:  283 out of:  288
iteraction no:  284 out of:  288
iteraction no:  285 out of:  288
iteraction no:  286 out of:  288
iteraction

iteraction no:  216 out of:  288
iteraction no:  217 out of:  288
iteraction no:  218 out of:  288
iteraction no:  219 out of:  288
iteraction no:  220 out of:  288
iteraction no:  221 out of:  288
iteraction no:  222 out of:  288
iteraction no:  223 out of:  288
iteraction no:  224 out of:  288
iteraction no:  225 out of:  288
iteraction no:  226 out of:  288
iteraction no:  227 out of:  288
iteraction no:  228 out of:  288
iteraction no:  229 out of:  288
iteraction no:  230 out of:  288
iteraction no:  231 out of:  288
iteraction no:  232 out of:  288
iteraction no:  233 out of:  288
iteraction no:  234 out of:  288
iteraction no:  235 out of:  288
iteraction no:  236 out of:  288
iteraction no:  237 out of:  288
iteraction no:  238 out of:  288
iteraction no:  239 out of:  288
iteraction no:  240 out of:  288
iteraction no:  241 out of:  288
iteraction no:  242 out of:  288
iteraction no:  243 out of:  288
iteraction no:  244 out of:  288
iteraction no:  245 out of:  288
iteraction

iteraction no:  177 out of:  288
iteraction no:  178 out of:  288
iteraction no:  179 out of:  288
iteraction no:  180 out of:  288
iteraction no:  181 out of:  288
iteraction no:  182 out of:  288
iteraction no:  183 out of:  288
iteraction no:  184 out of:  288
iteraction no:  185 out of:  288
iteraction no:  186 out of:  288
iteraction no:  187 out of:  288
iteraction no:  188 out of:  288
iteraction no:  189 out of:  288
iteraction no:  190 out of:  288
iteraction no:  191 out of:  288
iteraction no:  192 out of:  288
iteraction no:  193 out of:  288
iteraction no:  194 out of:  288
iteraction no:  195 out of:  288
iteraction no:  196 out of:  288
iteraction no:  197 out of:  288
iteraction no:  198 out of:  288
iteraction no:  199 out of:  288
iteraction no:  200 out of:  288
iteraction no:  201 out of:  288
iteraction no:  202 out of:  288
iteraction no:  203 out of:  288
iteraction no:  204 out of:  288
iteraction no:  205 out of:  288
iteraction no:  206 out of:  288
iteraction

iteraction no:  137 out of:  288
iteraction no:  138 out of:  288
iteraction no:  139 out of:  288
iteraction no:  140 out of:  288
iteraction no:  141 out of:  288
iteraction no:  142 out of:  288
iteraction no:  143 out of:  288
iteraction no:  144 out of:  288
iteraction no:  145 out of:  288
iteraction no:  146 out of:  288
iteraction no:  147 out of:  288
iteraction no:  148 out of:  288
iteraction no:  149 out of:  288
iteraction no:  150 out of:  288
iteraction no:  151 out of:  288
iteraction no:  152 out of:  288
iteraction no:  153 out of:  288
iteraction no:  154 out of:  288
iteraction no:  155 out of:  288
iteraction no:  156 out of:  288
iteraction no:  157 out of:  288
iteraction no:  158 out of:  288
iteraction no:  159 out of:  288
iteraction no:  160 out of:  288
iteraction no:  161 out of:  288
iteraction no:  162 out of:  288
iteraction no:  163 out of:  288
iteraction no:  164 out of:  288
iteraction no:  165 out of:  288
iteraction no:  166 out of:  288
iteraction

iteraction no:  98 out of:  288
iteraction no:  99 out of:  288
iteraction no:  100 out of:  288
iteraction no:  101 out of:  288
iteraction no:  102 out of:  288
iteraction no:  103 out of:  288
iteraction no:  104 out of:  288
iteraction no:  105 out of:  288
iteraction no:  106 out of:  288
iteraction no:  107 out of:  288
iteraction no:  108 out of:  288
iteraction no:  109 out of:  288
iteraction no:  110 out of:  288
iteraction no:  111 out of:  288
iteraction no:  112 out of:  288
iteraction no:  113 out of:  288
iteraction no:  114 out of:  288
iteraction no:  115 out of:  288
iteraction no:  116 out of:  288
iteraction no:  117 out of:  288
iteraction no:  118 out of:  288
iteraction no:  119 out of:  288
iteraction no:  120 out of:  288
iteraction no:  121 out of:  288
iteraction no:  122 out of:  288
iteraction no:  123 out of:  288
iteraction no:  124 out of:  288
iteraction no:  125 out of:  288
iteraction no:  126 out of:  288
iteraction no:  127 out of:  288
iteraction n

iteraction no:  67 out of:  288
iteraction no:  68 out of:  288
iteraction no:  69 out of:  288
iteraction no:  70 out of:  288
iteraction no:  71 out of:  288
iteraction no:  72 out of:  288
iteraction no:  73 out of:  288
iteraction no:  74 out of:  288
iteraction no:  75 out of:  288
iteraction no:  76 out of:  288
iteraction no:  77 out of:  288
iteraction no:  78 out of:  288
iteraction no:  79 out of:  288
iteraction no:  80 out of:  288
iteraction no:  81 out of:  288
iteraction no:  82 out of:  288
iteraction no:  83 out of:  288
iteraction no:  84 out of:  288
iteraction no:  85 out of:  288
iteraction no:  86 out of:  288
iteraction no:  87 out of:  288
iteraction no:  88 out of:  288
iteraction no:  89 out of:  288
iteraction no:  90 out of:  288
iteraction no:  91 out of:  288
iteraction no:  92 out of:  288
iteraction no:  93 out of:  288
iteraction no:  94 out of:  288
iteraction no:  95 out of:  288
iteraction no:  96 out of:  288
iteraction no:  97 out of:  288
iteracti

iteraction no:  42 out of:  288
iteraction no:  43 out of:  288
iteraction no:  44 out of:  288
iteraction no:  45 out of:  288
iteraction no:  46 out of:  288
iteraction no:  47 out of:  288
iteraction no:  48 out of:  288
iteraction no:  49 out of:  288
iteraction no:  50 out of:  288
iteraction no:  51 out of:  288
iteraction no:  52 out of:  288
iteraction no:  53 out of:  288
iteraction no:  54 out of:  288
iteraction no:  55 out of:  288
iteraction no:  56 out of:  288
iteraction no:  57 out of:  288
iteraction no:  58 out of:  288
iteraction no:  59 out of:  288
iteraction no:  60 out of:  288
iteraction no:  61 out of:  288
iteraction no:  62 out of:  288
iteraction no:  63 out of:  288
iteraction no:  64 out of:  288
iteraction no:  65 out of:  288
iteraction no:  66 out of:  288
iteraction no:  67 out of:  288
iteraction no:  68 out of:  288
iteraction no:  69 out of:  288
iteraction no:  70 out of:  288
iteraction no:  71 out of:  288
iteraction no:  72 out of:  288
iteracti

In [35]:
#dispatch rule with delay
env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"opt_disp",verbose=1)
results['dispatch delay']=(capacity,costs)

iteraction no:  0 out of:  288
iteraction no:  1 out of:  288
iteraction no:  2 out of:  288
iteraction no:  3 out of:  288
iteraction no:  4 out of:  288
iteraction no:  5 out of:  288
iteraction no:  6 out of:  288
iteraction no:  7 out of:  288
iteraction no:  8 out of:  288
iteraction no:  9 out of:  288
iteraction no:  10 out of:  288
iteraction no:  11 out of:  288
iteraction no:  12 out of:  288
iteraction no:  13 out of:  288
iteraction no:  14 out of:  288
iteraction no:  15 out of:  288
iteraction no:  16 out of:  288
iteraction no:  17 out of:  288
iteraction no:  18 out of:  288
iteraction no:  19 out of:  288
iteraction no:  20 out of:  288
iteraction no:  21 out of:  288
iteraction no:  22 out of:  288
iteraction no:  23 out of:  288
iteraction no:  24 out of:  288
iteraction no:  25 out of:  288
iteraction no:  26 out of:  288
iteraction no:  27 out of:  288
iteraction no:  28 out of:  288
iteraction no:  29 out of:  288
iteraction no:  30 out of:  288
iteraction no:  31

iteraction no:  253 out of:  288
iteraction no:  254 out of:  288
iteraction no:  255 out of:  288
iteraction no:  256 out of:  288
iteraction no:  257 out of:  288
iteraction no:  258 out of:  288
iteraction no:  259 out of:  288
iteraction no:  260 out of:  288
iteraction no:  261 out of:  288
iteraction no:  262 out of:  288
iteraction no:  263 out of:  288
iteraction no:  264 out of:  288
iteraction no:  265 out of:  288
iteraction no:  266 out of:  288
iteraction no:  267 out of:  288
iteraction no:  268 out of:  288
iteraction no:  269 out of:  288
iteraction no:  270 out of:  288
iteraction no:  271 out of:  288
iteraction no:  272 out of:  288
iteraction no:  273 out of:  288
iteraction no:  274 out of:  288
iteraction no:  275 out of:  288
iteraction no:  276 out of:  288
iteraction no:  277 out of:  288
iteraction no:  278 out of:  288
iteraction no:  279 out of:  288
iteraction no:  280 out of:  288
iteraction no:  281 out of:  288
iteraction no:  282 out of:  288
iteraction

iteraction no:  214 out of:  288
iteraction no:  215 out of:  288
iteraction no:  216 out of:  288
iteraction no:  217 out of:  288
iteraction no:  218 out of:  288
iteraction no:  219 out of:  288
iteraction no:  220 out of:  288
iteraction no:  221 out of:  288
iteraction no:  222 out of:  288
iteraction no:  223 out of:  288
iteraction no:  224 out of:  288
iteraction no:  225 out of:  288
iteraction no:  226 out of:  288
iteraction no:  227 out of:  288
iteraction no:  228 out of:  288
iteraction no:  229 out of:  288
iteraction no:  230 out of:  288
iteraction no:  231 out of:  288
iteraction no:  232 out of:  288
iteraction no:  233 out of:  288
iteraction no:  234 out of:  288
iteraction no:  235 out of:  288
iteraction no:  236 out of:  288
iteraction no:  237 out of:  288
iteraction no:  238 out of:  288
iteraction no:  239 out of:  288
iteraction no:  240 out of:  288
iteraction no:  241 out of:  288
iteraction no:  242 out of:  288
iteraction no:  243 out of:  288
iteraction

iteraction no:  174 out of:  288
iteraction no:  175 out of:  288
iteraction no:  176 out of:  288
iteraction no:  177 out of:  288
iteraction no:  178 out of:  288
iteraction no:  179 out of:  288
iteraction no:  180 out of:  288
iteraction no:  181 out of:  288
iteraction no:  182 out of:  288
iteraction no:  183 out of:  288
iteraction no:  184 out of:  288
iteraction no:  185 out of:  288
iteraction no:  186 out of:  288
iteraction no:  187 out of:  288
iteraction no:  188 out of:  288
iteraction no:  189 out of:  288
iteraction no:  190 out of:  288
iteraction no:  191 out of:  288
iteraction no:  192 out of:  288
iteraction no:  193 out of:  288
iteraction no:  194 out of:  288
iteraction no:  195 out of:  288
iteraction no:  196 out of:  288
iteraction no:  197 out of:  288
iteraction no:  198 out of:  288
iteraction no:  199 out of:  288
iteraction no:  200 out of:  288
iteraction no:  201 out of:  288
iteraction no:  202 out of:  288
iteraction no:  203 out of:  288
iteraction

iteraction no:  133 out of:  288
iteraction no:  134 out of:  288
iteraction no:  135 out of:  288
iteraction no:  136 out of:  288
iteraction no:  137 out of:  288
iteraction no:  138 out of:  288
iteraction no:  139 out of:  288
iteraction no:  140 out of:  288
iteraction no:  141 out of:  288
iteraction no:  142 out of:  288
iteraction no:  143 out of:  288
iteraction no:  144 out of:  288
iteraction no:  145 out of:  288
iteraction no:  146 out of:  288
iteraction no:  147 out of:  288
iteraction no:  148 out of:  288
iteraction no:  149 out of:  288
iteraction no:  150 out of:  288
iteraction no:  151 out of:  288
iteraction no:  152 out of:  288
iteraction no:  153 out of:  288
iteraction no:  154 out of:  288
iteraction no:  155 out of:  288
iteraction no:  156 out of:  288
iteraction no:  157 out of:  288
iteraction no:  158 out of:  288
iteraction no:  159 out of:  288
iteraction no:  160 out of:  288
iteraction no:  161 out of:  288
iteraction no:  162 out of:  288
iteraction

iteraction no:  92 out of:  288
iteraction no:  93 out of:  288
iteraction no:  94 out of:  288
iteraction no:  95 out of:  288
iteraction no:  96 out of:  288
iteraction no:  97 out of:  288
iteraction no:  98 out of:  288
iteraction no:  99 out of:  288
iteraction no:  100 out of:  288
iteraction no:  101 out of:  288
iteraction no:  102 out of:  288
iteraction no:  103 out of:  288
iteraction no:  104 out of:  288
iteraction no:  105 out of:  288
iteraction no:  106 out of:  288
iteraction no:  107 out of:  288
iteraction no:  108 out of:  288
iteraction no:  109 out of:  288
iteraction no:  110 out of:  288
iteraction no:  111 out of:  288
iteraction no:  112 out of:  288
iteraction no:  113 out of:  288
iteraction no:  114 out of:  288
iteraction no:  115 out of:  288
iteraction no:  116 out of:  288
iteraction no:  117 out of:  288
iteraction no:  118 out of:  288
iteraction no:  119 out of:  288
iteraction no:  120 out of:  288
iteraction no:  121 out of:  288
iteraction no:  12

iteraction no:  62 out of:  288
iteraction no:  63 out of:  288
iteraction no:  64 out of:  288
iteraction no:  65 out of:  288
iteraction no:  66 out of:  288
iteraction no:  67 out of:  288
iteraction no:  68 out of:  288
iteraction no:  69 out of:  288
iteraction no:  70 out of:  288
iteraction no:  71 out of:  288
iteraction no:  72 out of:  288
iteraction no:  73 out of:  288
iteraction no:  74 out of:  288
iteraction no:  75 out of:  288
iteraction no:  76 out of:  288
iteraction no:  77 out of:  288
iteraction no:  78 out of:  288
iteraction no:  79 out of:  288
iteraction no:  80 out of:  288
iteraction no:  81 out of:  288
iteraction no:  82 out of:  288
iteraction no:  83 out of:  288
iteraction no:  84 out of:  288
iteraction no:  85 out of:  288
iteraction no:  86 out of:  288
iteraction no:  87 out of:  288
iteraction no:  88 out of:  288
iteraction no:  89 out of:  288
iteraction no:  90 out of:  288
iteraction no:  91 out of:  288
iteraction no:  92 out of:  288
iteracti

iteraction no:  33 out of:  288
iteraction no:  34 out of:  288
iteraction no:  35 out of:  288
iteraction no:  36 out of:  288
iteraction no:  37 out of:  288
iteraction no:  38 out of:  288
iteraction no:  39 out of:  288
iteraction no:  40 out of:  288
iteraction no:  41 out of:  288
iteraction no:  42 out of:  288
iteraction no:  43 out of:  288
iteraction no:  44 out of:  288
iteraction no:  45 out of:  288
iteraction no:  46 out of:  288
iteraction no:  47 out of:  288
iteraction no:  48 out of:  288
iteraction no:  49 out of:  288
iteraction no:  50 out of:  288
iteraction no:  51 out of:  288
iteraction no:  52 out of:  288
iteraction no:  53 out of:  288
iteraction no:  54 out of:  288
iteraction no:  55 out of:  288
iteraction no:  56 out of:  288
iteraction no:  57 out of:  288
iteraction no:  58 out of:  288
iteraction no:  59 out of:  288
iteraction no:  60 out of:  288
iteraction no:  61 out of:  288
iteraction no:  62 out of:  288
iteraction no:  63 out of:  288
iteracti

demand left:___ 260 orig demand___ 1812 total  charged___ 1553
not charged:___ 0 not ended:___ 0 out of:  179
uncharged capacity:___ 0
costs:___ 61.629


cost/charged capacity(KWh)__: 0.0512 cost/charged capacity no intraday__: 0.05 percentage charged capacity__: 0.93 volume charged__: 14444


In [36]:
pickle.dump(results, open('output files/results'+fileName, "wb" ))

In [37]:
env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"SAA",verbose=1)
results['SAA']=(capacity,costs)

iteraction no:  0 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  1 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  2 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  3 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  4 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  5 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  6 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  7 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  8 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  9 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  10 out of:  288
Status: Optimal
St

Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  86 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  87 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  88 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  89 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  90 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  91 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  92 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  93 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  94 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  95 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: 

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  171 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  172 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  173 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  174 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  175 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  176 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  177 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  178 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  179 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  180 out of:  288
Status: Optimal
Status: Optima

Status: Optimal
iteraction no:  255 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  256 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  257 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  258 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  259 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  260 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  261 out of:  288
not enough to sample
not sorted
iteraction no:  262 out of:  288
not enough to sample
not sorted
iteraction no:  263 out of:  288
not enough to sample
not sorted
iteraction no:  264 out of:  288
not enough to sample
not sorted
iteraction no:  265 out of:  288
not enough to sample
not sorted
iteraction no:  266 out of:  288
not enough to sample
not sorted
iteraction n

Status: Optimal
Status: Optimal
iteraction no:  60 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  61 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  62 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  63 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  64 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  65 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  66 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  67 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  68 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  69 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteracti

...

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  126 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  127 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  128 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  129 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  130 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  131 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  132 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  133 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  134 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  135 out of:  288
Status: Optimal
Status: Optima

Status: Optimal
iteraction no:  210 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  211 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  212 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  213 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  214 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  215 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  216 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  217 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  218 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  219 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  20 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  21 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  22 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  23 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  24 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  25 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  26 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  27 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  28 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  29 out of:  288
Status: Optimal
Status: Optimal
Status: 

Status: Optimal
Status: Optimal
iteraction no:  105 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  106 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  107 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  108 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  109 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  110 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  111 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  112 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  113 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  114 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optima

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  190 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  191 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  192 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  193 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  194 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  195 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  196 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  197 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  198 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  199 out of:  288
Status: Optimal
Status: Optima

In [38]:
pickle.dump(results, open('output files/results'+fileName, "wb" ))

In [39]:


env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"SAA_SL",verbose=1)
results['SAA_SL']=(capacity,costs)

iteraction no:  0 out of:  288
train regressor
train performance:  

C:\Users\larsr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



0.8343494696091627
test performance:  0.7923613877968158


C:\Users\larsr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  1 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  2 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  3 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  4 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  5 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  6 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  7 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  8 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  9 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  10 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
S

...

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  126 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  127 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  128 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  129 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  130 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  131 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  132 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  133 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  134 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  135 out of:  288
Status: Optimal
Status: Optima

Status: Optimal
iteraction no:  210 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  211 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  212 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  213 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  214 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  215 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  216 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  217 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  218 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  219 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  20 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  21 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  22 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  23 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  24 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  25 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  26 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  27 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  28 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  29 out of:  288
Status: Optimal
Status: Optimal
Status: 

Status: Optimal
Status: Optimal
iteraction no:  105 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  106 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  107 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  108 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  109 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  110 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  111 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  112 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  113 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  114 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optima

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  190 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  191 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  192 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  193 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  194 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  195 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  196 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  197 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  198 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  199 out of:  288
Status: Optimal
Status: Optima

In [40]:
pickle.dump(results, open('output files/results'+fileName, "wb" ))

In [ ]:
#SAA sample
env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
uncharged,capacity,costs=run(env,7,intervalLength,"SAA_Samp",verbose=1)
results['SAA_Sample']=(capacity,costs)

iteraction no:  0 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  1 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  2 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  3 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  4 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  5 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  6 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  7 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  8 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  9 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  10 out of:  288
Status: Optimal
St

Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  86 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  87 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  88 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  89 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  90 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  91 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  92 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  93 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  94 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  95 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: 

Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  171 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  172 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  173 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  174 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  175 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  176 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  177 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  178 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  179 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  180 out of:  288
Status: Optimal
Status: Optima

Status: Optimal
iteraction no:  255 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  256 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  257 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  258 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  259 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  260 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  261 out of:  288
not enough to sample
not sorted
iteraction no:  262 out of:  288
not enough to sample
not sorted
iteraction no:  263 out of:  288
not enough to sample
not sorted
iteraction no:  264 out of:  288
not enough to sample
not sorted
iteraction no:  265 out of:  288
not enough to sample
not sorted
iteraction no:  266 out of:  288
not enough to sample
not sorted
iteraction n

Status: Optimal
Status: Optimal
iteraction no:  60 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  61 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  62 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  63 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  64 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  65 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  66 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  67 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  68 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  69 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteracti

Status: Optimal
Status: Optimal
iteraction no:  145 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  146 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  147 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  148 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  149 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  150 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  151 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  152 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  153 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optimal
iteraction no:  154 out of:  288
Status: Optimal
Status: Optimal
Status: Optimal
Status: Optima

In [ ]:
pickle.dump(results, open('output files/results'+fileName, "wb" ))

# Other

In [16]:
#total demand
totalDemand=False
if totalDemand:
    for key in demand1.keys():
        demand1[key]=(np.ones(288)*1000000,demand1[key][1])
    env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
    uncharged,capacity,costs=run(env,7,intervalLength,"priority",verbose=1)
    print(capacity)

In [17]:
arrivFirst=False
caps=[]
if arrivFirst:
    surs=[0.9,0.95,1,1.05,1.1,1.15,1.2,1.25]
    for sur in surs:
        fileName=str(sur).replace('.','_')
        demand1= pickle.load( open("source_solution/source_orig_"+fileName, "rb" ))

        env=Environment(df[df['days_for_aggregation']>=35],demand=demand1,prices=priceDf,intraPrices=intraDf,trainDf=trainDf)
        uncharged,capacity,costs=run(env,7,intervalLength,"arrival",verbose=0)
        caps.append(capacity)
        print(capacity)


In [18]:
reg=train_rf_regression(trainDf)

train regressor


C:\Users\larsr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



train performance:  0.8333697877622248
test performance:  0.7928884121695197


C:\Users\larsr\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning:

The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.



In [19]:
dataSupv=trainDf[['arrivaltime_no','duetime_no','time_till_due','MaxPower(KW)','weekday','TotalEnergy(KWh)']]
X=dataSupv.drop('TotalEnergy(KWh)',axis=1)

In [24]:
import plotly.graph_objects as go

fig = go.Figure()

    

subDf=trainDf
subDf['predEnergy']=reg.predict(X)
subDf=trainDf.sort_values(by='TotalEnergy(KWh)')
subDf
fig.add_trace(go.Scatter(y=subDf['TotalEnergy(KWh)'],
                    name='Total Energy'))
fig.add_trace(go.Scatter(y=subDf['predEnergy'],
                    name='Predicted Total Energy',opacity=0.5))
fig.update_layout(
    title="Regression analysis Energy volume individual EV's (R^2=0.79)",
    yaxis_title="Total Energy (KWh)",
    xaxis_title="Observations")
fig.show()

C:\Users\larsr\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [ ]:
fileName=str(1).replace('.','_')
demand1= pickle.load( open("source_solution/source_orig_"+fileName, "rb" ))

In [ ]:
df.keys()

In [22]:
df['days_for_aggregation'].unique()

array([ 3,  5,  0,  6,  4,  2,  1, 10,  7, 12,  8,  9, 13, 11, 18, 19, 17,
       20, 15, 16, 14, 23, 21, 22, 25, 27, 26, 24, 30, 29, 32, 28, 31, 34,
       33, 40, 41, 37, 39, 35, 38, 36], dtype=int64)

In [23]:
intervalLength=5
intervals=get_time_intervals(intervalLength)
truths=[]
preds=[]
for i in range(42):
    subDf=df[df['days_for_aggregation']==i]
    demand=list(calculate_demand(subDf,intervalLength,intervals,chargeType='asap'))
    weekday=df[df['days_for_aggregation']==i]['weekday'][0:1].values[0]
    estDem=demand1[weekday][0]
    truths+=demand
    preds+=list(estDem)
    

In [92]:
len(truths),len(preds)

(12096, 12096)

In [93]:
demandPredDf = pd.DataFrame(list(zip(truths, preds)), 
               columns =['truth_demand', 'pred_demand']) 

In [94]:
demandPredDf=demandPredDf.sort_values(by='truth_demand')

In [ ]:
42*

In [103]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=demandPredDf['truth_demand'],
                    name='Electricity demand'))
fig.add_trace(go.Scatter(y=demandPredDf['pred_demand'],
                    name='Predicted electricity demand',opacity=0.5))
fig.update_layout(
    title="Regression analysis total demand on 5 minute intervals (R^2=0.67)",
    yaxis_title="Electricity demand (KW)",
    xaxis_title="Observations")
fig.show()

In [97]:
from sklearn.metrics import r2_score
r2_score(demandPredDf['truth_demand'],demandPredDf['pred_demand'])

0.6688274655319644

In [116]:
np.std(df['TotalEnergy(KWh)'])/np.mean(df['TotalEnergy(KWh)'])

1.0485180307007136

In [117]:
np.std(demandPredDf['truth_demand'])/np.mean(demandPredDf['truth_demand'])

0.6748849069739146

In [118]:
np.std(df['duetime_no'])/np.mean(df['duetime_no'])

0.3607966230201093

In [ ]:
for i in range(7):
    estDem=demand1[i][0]
    subDf=df[df['weekday']==i]
    demand=list(calculate_demand(subDf,intervalLength,intervals,chargeType='asap'))
    
    weekday=df[df['days_for_aggregation']==i]['weekday'][0:1].values[0]
    estDem=demand1[weekday][0]
    truths+=demand
    preds+=list(estDem)

6204.845674011147